In [ ]:
import os
import json
import pandas as pd
import joblib
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from pathlib import Path

ROOT = Path(os.getcwd()).resolve().parent.parent
DATA = ROOT / "data"
TRAINED_DATA = DATA / "trained_data"


In [3]:
def remove_redundant_mean_features(features):
    """
    Loại bỏ các đặc trưng *_mean nếu đã có cả *_min và *_max tương ứng.

    Args:
        features (list): Danh sách tên các đặc trưng đã chọn.

    Returns:
        list: Danh sách đặc trưng sau khi loại bỏ các *_mean dư thừa.
    """
    features_set = set(features)
    cleaned_features = features.copy()

    # Tìm tất cả các nhóm có dạng <prefix>_min, <prefix>_mean, <prefix>_max
    for feature in features:
        if feature.endswith("_mean"):
            prefix = feature[:-5]  # Bỏ "_mean"
            min_feat = f"{prefix}_min"
            max_feat = f"{prefix}_max"
            if min_feat in features_set and max_feat in features_set:
                print(f"[INFO] Detected redundant features: {min_feat}, {feature}, {max_feat}. Removing {feature}.")
                cleaned_features.remove(feature)
                print(f"[INFO] Removed redundant mean feature: {feature}")

    return cleaned_features

In [7]:
METHOD = "SingleOutput"

# Load dữ liệu
df = pd.read_csv("../../data/processed/clean_hourly_weather.csv")

# Danh sách các biến mục tiêu
target_variables = [
    "temperature_2m",
    # "apparent_temperature",
    # "relative_humidity_2m",
    # "wind_speed_10m",
    # "wind_direction_10m_sin",
    # "wind_direction_10m_cos",
    # "rain",
    # "shortwave_radiation"
]

# Duyệt qua từng mục tiêu
for target_variable in target_variables:
    print(f"\n[INFO] Running features filtering for target: {target_variable}")
    # Tính tương quan với biến mục tiêu
    corr_with_target = df.corr(numeric_only=True)[target_variable].drop(target_variable)

    # Giữ lại các biến có tương quan < 0.99
    safe_features = corr_with_target[corr_with_target.abs() < 0.99].index.tolist()

    print(f"[INFO] The number of remaining features after proxying: {len(safe_features)}")

    # Chia dữ liệu train/test theo thời gian: 80% train, 20% test
    split_index = int(len(df) * 0.8)
    train_df = df.iloc[:split_index]
    test_df = df.iloc[split_index:]
    
    X_corr = df[safe_features]
    X_scaled = StandardScaler().fit_transform(X_corr)

    vif = pd.DataFrame()
    vif["feature"] = safe_features
    vif["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

    # Giữ lại những biến có VIF thấp hơn ngưỡng < 10
    vif_selected = vif[vif["VIF"] < 10]["feature"].tolist()

    # Gộp vào kết quả cuối cùng
    selected_features = sorted(set(vif_selected + ["season_sin", "season_cos"]))
    print("[INFO] Final selected features:", selected_features)

    # Tách lại theo đặc trưng được chọn cho train/test
    X_train = train_df[selected_features]
    X_test = test_df[selected_features]
    y_train = train_df[target_variable]
    y_test = test_df[target_variable]

    # Tạo pipeline chính
    preprocessor = ColumnTransformer(transformers=[("num", StandardScaler(), selected_features)])
    target_output_dir = TRAINED_DATA / METHOD.lower() / target_variable
    target_output_dir.mkdir(parents=True, exist_ok=True)
    
    # Lưu dữ liệu
    joblib.dump(X_train, target_output_dir / "X_train.pkl")
    joblib.dump(X_test, target_output_dir / "X_test.pkl")
    joblib.dump(y_train, target_output_dir / "y_train.pkl")
    joblib.dump(y_test, target_output_dir / "y_test.pkl")
    joblib.dump(preprocessor, target_output_dir / "preprocessor.pkl")
    with open(target_output_dir / "selected_features.json", "w") as f:
        json.dump(selected_features, f, indent=2)

    print(f"[INFO] Saved all artifacts to: {target_output_dir}")
    
    test_df = test_df.copy()
    test_df["date"] = df.iloc[split_index:]["date"].values
    test_df[target_variable] = y_test.values
    test_df.to_csv(target_output_dir / "test_df.csv", index=False)

print("\n✅ DONE: Feature selection and saving complete for all target variables.")


[INFO] Running features filtering for target: temperature_2m
[INFO] The number of remaining features after proxying: 47


c:\Users\ADMIN\MyProject\School_Projects\WeatherForecastML-DM\.venv\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


[INFO] Final selected features: ['boundary_layer_height', 'cloud_cover', 'cloud_cover_high', 'cloud_cover_low', 'cloud_cover_mid', 'is_day', 'season_cos', 'season_sin', 'soil_moisture_100_to_255cm', 'soil_moisture_28_to_100cm', 'sunshine_duration', 'wind_direction_100m_sin', 'wind_gusts_10m']
[INFO] Saved all artifacts to: C:\Users\ADMIN\MyProject\School_Projects\WeatherForecastML-DM\data\trained_data\singleoutput\temperature_2m

✅ DONE: Feature selection and saving complete for all target variables.


In [ ]:
METHOD = "MultiOutput"

df = pd.read_csv("../../data/processed/clean_hourly_weather.csv")

target_variables = [
    "temperature_2m",
    "apparent_temperature",
    "relative_humidity_2m",
    "wind_speed_10m",
    "wind_direction_10m_sin",
    "wind_direction_10m_cos",
    "rain",
    "shortwave_radiation"
]

numerical_cols = df.select_dtypes(include=["number"]).columns.tolist()
feature_candidates = [f for f in numerical_cols if f not in target_variables]

corr_matrix = df[target_variables + feature_candidates].corr(numeric_only=True)

# Lấy chỉ phần tương quan giữa đặc trưng và mục tiêu
corr_features_targets = corr_matrix.loc[feature_candidates, target_variables]

max_corr = corr_features_targets.abs().max(axis=1)

selected_features = max_corr[max_corr < 0.995].index.tolist()

X_scaled = StandardScaler().fit_transform(df[selected_features])
vif = pd.DataFrame()
vif["feature"] = selected_features
vif["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

# Loại các feature có VIF >= 10
vif_selected = vif[vif["VIF"] < 10]["feature"].tolist()

selected_features = sorted(set(vif_selected + ["season_sin", "season_cos"]))
print(f"[INFO] Detecting redundant features...")
selected_features = remove_redundant_mean_features(selected_features)
print("[INFO] Final selected features:", selected_features)

split_index = int(len(df) * 0.8)
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

# Tách lại theo đặc trưng được chọn cho train/test
X_train = train_df[selected_features]
X_test = test_df[selected_features]
y_train = train_df[target_variables]
y_test = test_df[target_variables]

# Tạo pipeline chính
preprocessor = ColumnTransformer(transformers=[("num", StandardScaler(), selected_features)])
target_output_dir = TRAINED_DATA / METHOD.lower()
target_output_dir.mkdir(parents=True, exist_ok=True)

# Lưu dữ liệu
joblib.dump(X_train, target_output_dir / "X_train.pkl")
joblib.dump(X_test, target_output_dir / "X_test.pkl")
joblib.dump(y_train, target_output_dir / "y_train.pkl")
joblib.dump(y_test, target_output_dir / "y_test.pkl")
joblib.dump(preprocessor, target_output_dir / "preprocessor.pkl")
with open(target_output_dir / "selected_features.json", "w") as f:
    json.dump(selected_features, f, indent=2)

print(f"[INFO] Saved all artifacts to: {target_output_dir}")

test_df = test_df.copy()
test_df["date"] = df.iloc[split_index:]["date"].values
test_df[target_variables] = y_test.values
test_df.to_csv(target_output_dir / "test_df.csv", index=False)

[INFO] Detecting redundant features...
[INFO] Final selected features: ['boundary_layer_height', 'cloud_cover', 'cloud_cover_high', 'cloud_cover_low', 'cloud_cover_mid', 'is_day', 'precipitation', 'season_cos', 'season_sin', 'soil_moisture_100_to_255cm', 'soil_moisture_28_to_100cm', 'sunshine_duration', 'total_column_integrated_water_vapour', 'wind_direction_100m_cos', 'wind_direction_100m_sin', 'wind_gusts_10m', 'wind_speed_100m']
[INFO] Saved all artifacts to: C:\Users\ADMIN\MyProject\School_Projects\WeatherForecastML-DM\data\trained_data\hourly
